## 标记是一个很重要的工作

https://github.com/wkentaro/labelme

1. 需要标记的放在一个文件夹下面
2. label.txt 文件，记录类别名称，每行一个类别名称，第一行是背景类，一般不需要标记
``` 
__ignore__
_background_
cat
```
3. 使用 **labelme_json_to_dataset** 或者 **label2voc** 工具将标记文件转换为我们需要的格式
4. 运行以下脚本自动启动 Labelme
```bash
labelme --labels labels.txt --nodata
```
5. 点我们击左侧的 Open Dir 打开图片文件夹，选择一张图片进行标记
6. 点击左侧的 Create Polygons Tool 进行标记。当标记完成后，我们需要保存一下，保存之后会生成标记好的 json 文件。
7. 执行下面bash脚本，将 json 文件转换为我们需要的 mask 图像 位置在exameple/instance_segmentation/labelme2voc.py
```bash
python label2voc.py cats cats_output --label label.txt 
```

## 数据读取

完成了标记工作之后，我们就要用 PyTorch 把这些数据给读入进来了，我们把数据相关的写在 dataset.py 中。

具体还是和之前讲的一样，要继承 Dataset 类，然后实现 __init__、__len__ 和 __getitem__ 方法。

In [63]:
import os
import torch
import numpy as np
import argparse
import json

from tqdm import tqdm
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from PIL import Image

class CatSegmentationDataset(Dataset):
    in_channels = 3
    out_channels = 1

    def __init__(self, images_dir, images_size=32):
        print("Reading data from:", images_dir)
        image_root_path =  images_dir + os.sep + "JPEGImages"
        mask_root_path = images_dir + os.sep + "SegmentationClassPNG"

        self.image_slices = []
        self.mask_slices = []

        for im_name in os.listdir(image_root_path):
            mask_name = im_name.split(".")[0] + ".png"

            # os.sep is the system path separator, e.g., '/' for Linux and '\' for Windows
            image_path = image_root_path + os.sep + im_name
            mask_path = mask_root_path + os.sep + mask_name

            im = np.asarray(Image.open(image_path).resize((images_size, images_size)))
            mask = np.asarray(Image.open(mask_path).resize((images_size, images_size)))

            self.image_slices.append(im / 255.0)
            self.mask_slices.append(mask)

    def __len__(self):
        return len(self.image_slices)

    def __getitem__(self, idx):
        image = self.image_slices[idx]
        mask = self.mask_slices[idx]

        image =  image.transpose((2, 0, 1))  # HWC -> CHW
        mask = mask[np.newaxis, ...]  # HW -> CHW

        image = image.astype(np.float32)
        mask = mask.astype(np.float32)

        return image, mask

In [64]:

def data_loaders(args):
    dataset_train = CatSegmentationDataset(
        images_dir=args.images,
        images_size=args.image_size
    )

    loader_train = DataLoader(
        dataset_train,
        batch_size=args.batch_size,
        shuffle=True,
        num_workers=args.num_workers,
    )

    return loader_train

## 设计模型

![image10.png](mdfiles/image10.png)


1. 图中横向蓝色的箭头，它们都是重复的相同结构，都是由两个 3x3 的卷积层组合而成的，在每层卷积之后会跟随一个 BN 层与 ReLU 的激活层

2. 注意绿色箭头的功能，是表示上采样操作，使用的是转置卷积来实现的

3. 红色箭头表示max pooling 下采样操作

4. 最后输出层使用是二分类因此使用 1x1 的卷积将通道数变为 2 ！！！

5. 灰色表示跳跃连接，将编码器部分的特征图与解码器部分对应尺寸的特征图进行拼接

## 损失函数

Dice Loss

Dice 系数，常用于计算两个集合的相似度，取值范围在 0-1 之间

$$
\text { Dice }=\frac{2|P \cap G|}{|P|+|G|}
$$

GT 只有 0 和 1 两个值。当我们直接使用模型输出的预测概率而不是使用阈值将它们转换为二值 Mask 时，这种损失函数就被称为 Soft Dice Loss。此时，∣P∩G∣ 的值近似为 GT 与预测概率矩阵的点乘。

根据 Dice 系数我们就能设计出一种损失函数，也就是 Dice Loss。它的计算公式非常简单，如下所示。

$$
\text { Dice Loss }=1-\text { Dice }
$$

当预测值的 Mask 与 GT 越相似，损失就越小；当预测值的 Mask 与 GT 差异度越大，损失就越大

In [65]:
import torch.nn as nn

class DiceLoss(nn.Module):
    def __init__(self, class_weights=None, smooth=1.0):
        super(DiceLoss, self).__init__()
        self.smooth = float(smooth)

        if class_weights is not None:
            w = torch.tensor(class_weights, dtype=torch.float32)
            self.register_buffer("class_weights", w)
        else:
            self.register_buffer("class_weights", None)



    def forward(self, y_pred, y_true):
        # ones = torch.ones(1, device=y_pred.device, dtype=y_pred.dtype)
        assert y_pred.size() == y_true.size()
        y_pred = y_pred[:, 0].contiguous().view(-1)
        y_true = y_true[:, 0].contiguous().view(-1)
        intersection = (y_pred * y_true).sum()
        dsc = (2. * intersection + self.smooth) / (
            y_pred.sum() + y_true.sum() + self.smooth
        )
        return 1. - dsc

In [66]:
import importlib
from unet import UNet


def makedirs(args):
    os.makedirs(args.ckpts, exist_ok=True)
    os.makedirs(args.logs, exist_ok=True)


def audit_devices(model):
    bad = []
    for name, p in model.named_parameters():
        if p.device.type == "cpu":
            bad.append(("param", name, tuple(p.shape), p.device))
    for name, b in model.named_buffers():
        if b.device.type == "cpu":
            bad.append(("buffer", name, tuple(b.shape), b.device))
    return bad


def main(args):
    makedirs(args)
    device = torch.device("cpu" if not torch.cuda.is_available() else args.device)

    loader_train = data_loaders(args)

    unet = UNet(in_channels=CatSegmentationDataset.in_channels, out_channels=CatSegmentationDataset.out_channels)
    unet = unet.to(device)

    problems = audit_devices(unet)
    if problems:
        print("Still on CPU before forward():")
        for kind, name, shape, dev in problems:
            print(f" - {kind:6s} {name:40s} {shape} @ {dev}")
    else:
        print("All registered params/buffers moved to", device)

    dsc_loss = DiceLoss().to(device)

    optimizer = optim.Adam(unet.parameters(), lr=args.lr)

    loss_train = []

    step = 0

    for epoch in tqdm(range(args.epochs), total=args.epochs):
        unet.train()
        for batch_idx, (data, target) in enumerate(loader_train):
            step += 1
            data   = torch.as_tensor(data, device=device)
            target = torch.as_tensor(target, device=device)

            y_pred = unet(data)
            optimizer.zero_grad()
            loss = dsc_loss(y_pred, target)

            loss_train.append(loss.item())
            loss.backward()
            optimizer.step()

            if (step) % 10 == 0:
                print('Step', step, 'Training Loss:', np.mean(loss_train[-10:]))

        torch.save(unet, args.ckpts + '/unet_epoch_{}.pth'.format(epoch))

In [67]:
# 在 Notebook 中使用 argparse 不方便，这里直接构造一个默认参数对象
args = argparse.Namespace(
    batch_size=16,      # Batch Size
    epochs=10,         # Epoch number
    lr=0.0001,         # Learning rate
    device="cuda:0",   # Device for training (auto-falls back to CPU if no CUDA)
    num_workers=4,     # Dataloader workers
    ckpts="./ckpts/work2",   # folder to save weights
    logs="./logs",     # folder to save logs
    images="./data/work2",   # root folder with images (should contain JPEGImages & SegmentationClassPNG)
    image_size=256     # target input image size
)

# 运行训练
main(args)

Reading data from: ./data/work2
All registered params/buffers moved to cuda:0


  0%|          | 0/10 [00:00<?, ?it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument weight in method wrapper_CUDA__cudnn_batch_norm)

## 几个地方需要注意换到device上

1. model = model.to(device=device)
2. data   = data.to(device=device, non_blocking=True)
3. target = target.to(device=device, non_blocking=True)
4. loss = loss.to(device=device)

注意在网络不建议使用没有在init里面初始化的操作放在forward里面

使用 import torch.nn.functional as F, 然后在 forward 里面使用 F.xxx() 的方式调用 例如 F.relu()  F.softmax()  F.conv2d() F.max_pool2d()

## 评价指标

常用的评价指标是 mIoU。mIoU 全称为 mean Intersection over Union，即平均交并比。交并比是真实值和预测值的交集和并集之比。

![image12.png](mdfiles/image12.png)




## 新图片测试

没学过的果然不行

![output_overlay.png](output_overlay.png)